<a href="https://colab.research.google.com/github/Priyo-prog/Movie-Recommendation-using-Boltzmann-Machine/blob/main/Boltzmann_Machine_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie recommendation system using Boltzmann Machine

* Use the data from Movielens to analyze the data on movies https://grouplens.org/datasets/movielens/

* Use Torch Libraries to build a Boltzmann Machine

## Import important libraries

In [2]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Import the data

In [3]:
# Mount the drive amd import the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import the dataset
movies = pd.read_csv('/content/drive/MyDrive/Deep Learning/Boltzmann Machine Datasets/ml-1m/movies.dat', 
                     sep='::', header=None, engine='python', encoding='latin-1')

In [5]:
movies.head(5)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
